In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image_path = "intersecting.png"
img = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply bilateral filter to smooth the image while preserving edges
gray = cv2.bilateralFilter(gray, 10, 23, 3)

# Apply Laplacian edge detection
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = np.uint8(np.absolute(lap))

# Contour detection
contours, hierarchy = cv2.findContours(lap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Remove small objects by calculating the perimeter
perimeter_threshold = 60  # Adjust this threshold as needed
filtered_contours = [cnt for cnt in contours if cv2.arcLength(cnt, True) > perimeter_threshold]

# Draw contours on a white background
white_background = np.ones_like(img) * 255
contour_image = cv2.drawContours(white_background, filtered_contours, -1, (0, 0, 0), 2)

# Threshold
threshold_value = 106
_, thresh = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

# Remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# Invert the mask to create a mask for numbers
mask = cv2.bitwise_not(thresh)

# Use Shi-Tomasi corner detection
corners = cv2.goodFeaturesToTrack(mask, maxCorners=100, qualityLevel=0.01, minDistance=10)

if corners is not None:
    corners = np.int0(corners)

# Draw corners on the mask image
mask_with_corners = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
if corners is not None:
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(mask_with_corners, (x, y), 3, (0, 0, 255), -1)

# Count the number of corners detected and calculate the number of bends
num_corners = len(corners) if corners is not None else 0
num_bends = num_corners -2 # Subtract 3 from the number of corners

# Plotting the results
plt.figure(figsize=(15, 4))

# Plot the original image
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(lap, cv2.COLOR_BGR2RGB))
plt.title('Edge Detection')

# Plot the edge detection result
plt.subplot(1, 3, 2)
plt.imshow(mask, cmap='gray')
plt.title('Edge Detection')

# Plot the mask image with detected corners
plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(mask_with_corners, cv2.COLOR_BGR2RGB))
plt.title(f'Number of bends: {num_bends}')

# Show the plots
plt.show()

# Clean up windows (useful for interactive environments)
cv2.destroyAllWindows()

# cv2.imwrite("st_CAB_output.jpg", mask)
# cv2.imwrite("st_CAB_bends.jpg", mask_with_corners)